# Independent Q-learning

> In this notebook, we implement the **[Independent Q learning]()** Multi Agent Reinforcement Leaning (MARL) algorithm in our environment. 


> Tutorial based on [IQL TorchRL Tutorial](https://github.com/pytorch/rl/blob/main/sota-implementations/multiagent/iql.py).

#### High-level overview of IQL algorithm

In IQL a centralized state-action value function is used, Q<sub>tot</sub>, and each agent α learns an individual action-value function Q<sub>α</sub>, independently.

### Simulation overview

> We simulate our environment with an initial population of **20 human agents**. These agents navigate the environment and eventually converge on the fastest path. After this convergence, we will transition **10 of these human agents** into **machine agents**, specifically autonomous vehicles (AVs), which will then employ the Independent Q learning reinforcement learning algorithm to further refine their learning.

#### Imported libraries

In [1]:
%run startup.py

In [2]:
import os
import torch

from tensordict.nn import TensorDictModule, TensorDictSequential
from torchrl.envs.libs.pettingzoo import PettingZooWrapper
from torchrl.envs.transforms import TransformedEnv, RewardSum
from torchrl.envs.utils import check_env_specs
from torch import nn
from torchrl._utils import logger as torchrl_logger
from torchrl.collectors import SyncDataCollector
from torchrl.data import TensorDictReplayBuffer
from torchrl.data.replay_buffers.samplers import SamplerWithoutReplacement
from torchrl.data.replay_buffers.storages import LazyTensorStorage
from torchrl.modules import EGreedyModule, QValueModule, SafeSequential
from torchrl.modules.models.multiagent import MultiAgentMLP
from torchrl.objectives import SoftUpdate, ValueEstimators, DQNLoss

from RouteRL import TrafficEnvironment

os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [3]:
device = (
    torch.device(0)
    if torch.cuda.is_available()
    else torch.device("cpu")
)

print("device is: ", device)

device is:  cpu


#### Hyperparameters setting

In [4]:
# Sampling
frames_per_batch = 100  # Number of team frames collected per training iteration
n_iters = 500  # Number of sampling and training iterations - the episodes the plotter plots
total_frames = frames_per_batch * n_iters

# Training
num_epochs = 1  # Number of optimization steps per training iteration 
minibatch_size = 16  # Size of the mini-batches in each optimization step
lr = 3e-3 # Learning rate
max_grad_norm = 5.0  # Maximum norm for the gradients
memory_size = 5_000  # Size of the replay buffer
tau =  0.05
gamma = 0.99  # discount factor
exploration_fraction = 1/3 # Fraction of frames over which the exploration rate is annealed

# Pretraining
pretrain_steps = 100

# Environment
env_params = {
    "agent_parameters" : {
        "num_agents" : 200,
        "new_machines_after_mutation": 50,
        "human_parameters" : {
            "model" : "w_avg"
        },
    },
    "simulator_parameters" : {
        "network_name" : "csomor"
    },  
    "plotter_parameters" : {
        "phases" : [0, pretrain_steps],
        "smooth_by" : 50,
    }
}

#### Environment initialization

> In this example, the environment initially contains only human agents.

In [5]:
env = TrafficEnvironment(seed=42, **env_params)
print(env)

[CONFIRMED] Environment variable exists: SUMO_HOME
[SUCCESS] Added module directory: /opt/homebrew/opt/sumo/share/sumo/tools
TrafficEnvironment with 200 agents.            
0 machines and 200 humans.            
Machines: []            
Humans: [Human 0, Human 1, Human 2, Human 3, Human 4, Human 5, Human 6, Human 7, Human 8, Human 9, Human 10, Human 11, Human 12, Human 13, Human 14, Human 15, Human 16, Human 17, Human 18, Human 19, Human 20, Human 21, Human 22, Human 23, Human 24, Human 25, Human 26, Human 27, Human 28, Human 29, Human 30, Human 31, Human 32, Human 33, Human 34, Human 35, Human 36, Human 37, Human 38, Human 39, Human 40, Human 41, Human 42, Human 43, Human 44, Human 45, Human 46, Human 47, Human 48, Human 49, Human 50, Human 51, Human 52, Human 53, Human 54, Human 55, Human 56, Human 57, Human 58, Human 59, Human 60, Human 61, Human 62, Human 63, Human 64, Human 65, Human 66, Human 67, Human 68, Human 69, Human 70, Human 71, Human 72, Human 73, Human 74, Human 75, Huma

> Reset the environment and the connection with SUMO

In [6]:
env.start()
env.reset()

 Retrying in 1 seconds


({}, {})

#### Human learning

In [7]:
for episode in range(pretrain_steps):
    env.step()

#### Mutation

> **Mutation**: a portion of human agents are converted into machine agents (autonomous vehicles).

In [8]:
env.mutation()
print(env)

TrafficEnvironment with 200 agents.            
50 machines and 150 humans.            
Machines: [Machine 3, Machine 4, Machine 24, Machine 27, Machine 32, Machine 33, Machine 50, Machine 51, Machine 52, Machine 55, Machine 57, Machine 58, Machine 62, Machine 64, Machine 65, Machine 68, Machine 72, Machine 73, Machine 77, Machine 83, Machine 85, Machine 86, Machine 89, Machine 90, Machine 93, Machine 94, Machine 98, Machine 99, Machine 100, Machine 105, Machine 107, Machine 114, Machine 118, Machine 122, Machine 125, Machine 130, Machine 131, Machine 146, Machine 151, Machine 152, Machine 155, Machine 157, Machine 165, Machine 168, Machine 178, Machine 181, Machine 182, Machine 185, Machine 187, Machine 191]            
Humans: [Human 0, Human 1, Human 2, Human 5, Human 6, Human 7, Human 8, Human 9, Human 10, Human 11, Human 12, Human 13, Human 14, Human 15, Human 16, Human 17, Human 18, Human 19, Human 20, Human 21, Human 22, Human 23, Human 25, Human 26, Human 28, Human 29, Human 30

> Create a group that contains all the machine (RL) agents.

>  **Hint:** the agents aren't competely independent in this example.

#### PettingZoo environment wrapper

In [9]:
group = {'agents': [str(machine.id) for machine in env.machine_agents]}

env = PettingZooWrapper(
    env=env,
    use_mask=True,
    categorical_actions=True,
    done_on_any = False,
    group_map=group,
    device=device
)

> The environment is defined by a series of metadata that describe what can be expected during its execution. 

There are four specs to look at:

- <code style="color:white">action_spec</code> defines the action space;

- <code style="color:white">reward_spec</code> defines the reward domain;

- <code style="color:white">done_spec</code> defines the done domain;

- <code style="color:white">observation_spec</code> which defines the domain of all other outputs from environment steps;

In [10]:
"""
print("action_spec:", env.full_action_spec, "\n\n")
print("reward_spec:", env.full_reward_spec, "\n\n")
print("done_spec:", env.full_done_spec, "\n\n")
print("observation_spec:", env.observation_spec, "\n\n")
"""
;

''

#### Transforms

> We can append any TorchRL transform we need to our environment. These will modify its input/output in some desired way. In multi-agent contexts, it is paramount to provide explicitly the keys to modify.



Here we instatiate a <code style="color:white">RewardSum</code> transformer that will sum rewards over episode.

In [11]:
env = TransformedEnv(
    env,
    RewardSum(in_keys=[env.reward_key], out_keys=[("agents", "episode_reward")]),
)

The <code style="color:white">check_env_specs()</code> function runs a small rollout and compared it output against the environment specs. It will raise an error if the specs aren't properly defined.

In [12]:
check_env_specs(env)
env.reset()

2025-02-03 23:34:44,081 [torchrl][INFO] check_env_specs succeeded!


TensorDict(
    fields={
        agents: TensorDict(
            fields={
                done: Tensor(shape=torch.Size([50, 1]), device=cpu, dtype=torch.bool, is_shared=False),
                episode_reward: Tensor(shape=torch.Size([50, 1]), device=cpu, dtype=torch.float32, is_shared=False),
                mask: Tensor(shape=torch.Size([50]), device=cpu, dtype=torch.bool, is_shared=False),
                observation: Tensor(shape=torch.Size([50, 3]), device=cpu, dtype=torch.float32, is_shared=False),
                terminated: Tensor(shape=torch.Size([50, 1]), device=cpu, dtype=torch.bool, is_shared=False),
                truncated: Tensor(shape=torch.Size([50, 1]), device=cpu, dtype=torch.bool, is_shared=False)},
            batch_size=torch.Size([50]),
            device=cpu,
            is_shared=False),
        done: Tensor(shape=torch.Size([1]), device=cpu, dtype=torch.bool, is_shared=False),
        terminated: Tensor(shape=torch.Size([1]), device=cpu, dtype=torch.bool, is_

#### Policy network

> Instantiate an `MPL` that can be used in multi-agent contexts.

In [13]:
net = MultiAgentMLP(
        n_agent_inputs=env.observation_spec["agents", "observation"].shape[-1],
        n_agent_outputs=env.action_spec.space.n,
        n_agents=env.n_agents,
        centralised=False,
        share_params=False,
        device=device,
        depth=2,
        num_cells=32,
        activation_class=nn.ReLU,
    )

> The neural network is wrapped in a `TensorDictModule`, which is responsible for managing the input and output interactions with the tensordict. Specifically, the module reads from the specified `in_keys`, processes the inputs through the neural network, and writes the resulting outputs to the defined `out_keys`. 

In [14]:
module = TensorDictModule(
        net, in_keys=[("agents", "observation")], out_keys=[("agents", "action_value")]
)

> **`QValueModule`** takes a tensor as input, which contains the `Q-values` (these values indicate how good it is to take each action in the given state). It identifies the action with the highest `Q-values` using the `argmax` operation.

In [15]:
value_module = QValueModule(
    action_value_key=("agents", "action_value"),
    out_keys=[
        env.action_key,
        ("agents", "action_value"),
        ("agents", "chosen_action_value"),
    ],
    spec=env.action_spec,
    action_space=None,
)

> **`SafeSequential`** is a `TensordictModule` that will concatenate the parameter lists in a single list.

In [16]:
qnet = SafeSequential(module, value_module)

> In the already made `Q network` the **`Epsilon-Greedy exploration module`** is added. This module randomly updates the actions in a tensordict given an epsilon greedy exploration strategy.

In [17]:
qnet_explore = TensorDictSequential(
    qnet,
    EGreedyModule(
        eps_init=0.99,
        eps_end=0,
        annealing_num_steps=int(total_frames * exploration_fraction),
        action_key=env.action_key,
        spec=env.action_spec,
    ),
)

#### Collector

Collectors perform the following operations:

1. **Reset Environment**: Initialize the environment.
2. **Compute Action**: Determine the next action using the policy and the latest observation.
3. **Execute Step**: Step through the environment with the computed action.

These operations repeat until the environment signals to stop.

In [18]:
collector = SyncDataCollector(
        env,
        qnet_explore,
        device=device,
        storing_device=device,
        frames_per_batch=frames_per_batch,
        total_frames=total_frames,
    )

#### Replay buffer

> In an off-policy setting, the replay buffer exceeds the number of frames utilized for policy updates, allowing agents to learn from previous rollouts as well.



In [19]:
replay_buffer = TensorDictReplayBuffer(
        storage=LazyTensorStorage(memory_size, device=device),
        sampler=SamplerWithoutReplacement(),
        batch_size=minibatch_size,
    )

#### DQN loss function

In [20]:
loss_module = DQNLoss(qnet, delay_value=True)

loss_module.set_keys(
        action_value=("agents", "action_value"),
        action=env.action_key,
        value=("agents", "chosen_action_value"),
        reward=env.reward_key,
        done=("agents", "done"),
        terminated=("agents", "terminated"),
)

loss_module.make_value_estimator(ValueEstimators.TD0, gamma=gamma)
target_net_updater = SoftUpdate(loss_module, eps=1-tau)

optim = torch.optim.Adam(loss_module.parameters(), lr)

#### Training loop

In [21]:
for i, tensordict_data in enumerate(collector):
    torchrl_logger.info(f"\nIteration {i}")

    current_frames = tensordict_data.numel()
    data_view = tensordict_data.reshape(-1)
    replay_buffer.extend(data_view)
    
    training_tds = []

    ## Update the policies of the learning agents
    for _ in range(num_epochs):
        for _ in range(frames_per_batch // minibatch_size):
            subdata = replay_buffer.sample()
            loss_vals = loss_module(subdata)
            training_tds.append(loss_vals.detach())

            loss_value = loss_vals["loss"]

            loss_value.backward()

            #total_norm = torch.nn.utils.clip_grad_norm_(loss_module.parameters(), max_grad_norm)
            #training_tds[-1].set("grad_norm", total_norm.mean())

            optim.step()
            optim.zero_grad()
        target_net_updater.step()##

    qnet_explore[1].step(frames=current_frames)  # Update exploration annealing
    collector.update_policy_weights_()
    
    training_tds = torch.stack(training_tds) 

collector.shutdown()

2025-02-03 23:34:46,566 [torchrl][INFO] 
Iteration 0
2025-02-03 23:34:48,639 [torchrl][INFO] 
Iteration 1
2025-02-03 23:34:50,718 [torchrl][INFO] 
Iteration 2
2025-02-03 23:34:52,804 [torchrl][INFO] 
Iteration 3
2025-02-03 23:34:54,894 [torchrl][INFO] 
Iteration 4
2025-02-03 23:34:57,000 [torchrl][INFO] 
Iteration 5
2025-02-03 23:34:59,090 [torchrl][INFO] 
Iteration 6
2025-02-03 23:35:01,199 [torchrl][INFO] 
Iteration 7
2025-02-03 23:35:03,269 [torchrl][INFO] 
Iteration 8
2025-02-03 23:35:05,337 [torchrl][INFO] 
Iteration 9
2025-02-03 23:35:07,424 [torchrl][INFO] 
Iteration 10
2025-02-03 23:35:09,500 [torchrl][INFO] 
Iteration 11
2025-02-03 23:35:11,580 [torchrl][INFO] 
Iteration 12
2025-02-03 23:35:13,652 [torchrl][INFO] 
Iteration 13
2025-02-03 23:35:15,715 [torchrl][INFO] 
Iteration 14
2025-02-03 23:35:17,801 [torchrl][INFO] 
Iteration 15
2025-02-03 23:35:19,879 [torchrl][INFO] 
Iteration 16
2025-02-03 23:35:21,984 [torchrl][INFO] 
Iteration 17
2025-02-03 23:35:24,061 [torchrl][INFO

In [22]:
env.plot_results()